![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 2: Basic Chisel

#### Written by Stevo Bailey, Edward Wang, and Richard Lin
stevo@berkeley.edu, edwardw@berkeley.edu, richard.lin@berkeley.edu

## Introduction

Now that you're familiar with Scala, it's time to learn Chisel. Chisel is a hardware construction language (HCL), similar to a hardware design language but embedded in a higher level, functional programming language. The details and benefits of an HCL will be covered in later modules. This module introduces basic Chisel, which is quite similar to Verilog. Once the basics are understood, you can use them to construct parameterized generators and take full advantage of the language.

Throughout this module, the goal will be to produce a linear feedback shift register (LFSR) in hardware. If you are unfamiliar with LFSRs, skim the [Wikipedia page](https://en.wikipedia.org/wiki/Linear-feedback_shift_register) first. To do this, we will need to understand how combinational logic, sequential logic, operators, and data types are all handled in Chisel.

At a high level, the basic 4-bit LFSR circuit we will be building will look like:

![LFSR-4](images/lfsr4.svg)

## Setup

Download and import Chisel.

In [24]:
import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

def getVerilog[T <: Module](gen: => T): String = {
  chisel3.Driver.execute(Array[String](), {() => gen}) match {
    case ChiselExecutionSuccess(_, _, Some(firrtl.FirrtlExecutionSuccess(_, verilog))) => verilog
  }
}

import $ivy.$                                                 

import $ivy.$                                                          

import $ivy.$                                                

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}


defined function getVerilog

# A Simple Example
This section will present the simple hardware module, a simple test, and how to run it. It will contain many things that you will not understand, and that is ok. We want you to take away the broad strokes, so you can continually return to this complete working example to reinforce what you've learned.

## A Simple Module

Like Verilog, we can declare module definitions in Chisel. The following example is a Chisel `Module`, `Simple` that has one input, `in`, and one output, `out`, and inside it combinationally connects `in` and `out`, so `in` drives `out`.

What is neat about HCL's is we can use the underlying programming language as a scripting language. For example, after declaring our Chisel module, we then use Scala to call the Chisel compiler to translate Chisel `Simple` into Verilog `Simple`. This process is called ***elaboration***.

In [33]:
// Chisel Code: Declare a new module definition
class Simple extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code, it is very complicated Scala
println(getVerilog(new Simple))

[info] [0.000] Elaborating design...
[info] [0.001] Done elaborating.
Total FIRRTL Compile Time: 5.3 ms
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif

module cmd32WrapperHelperSimple(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in;
endmodule



defined class Simple

Note that the Name of our module is `cmd<#>WrapperHelperSimple`, which is an artifact of running this tutorial in Jupyter. In your normal code, it should have the name Simple. This is an important lesson though - although Chisel does its best to preserve the names of your modules and other hardware components, sometimes it fails to do so.

## A Simple Tester

No hardware module should be complete without a tester. The following example is a Chisel test harness `SimpleTester` that passes values to an instance of `Simple`'s input port `in`, and checks that the same value is seen on the output port `out`.

Like before, we also use Scala to script our execution of the test.

In [9]:
// Chisel Code: Declare a new tester for Simple modules
class SimpleTester(c: Simple) extends PeekPokeTester(c) {
  poke(c.io.in, 0)     // Set our input to value 0
  expect(c.io.out, 0)  // Assert that the output correctly has 0
  poke(c.io.in, 1)     // Set our input to value 1
  expect(c.io.out, 1)  // Assert that the output correctly has 1
  poke(c.io.in, 2)     // Set our input to value 2
  expect(c.io.out, 2)  // Assert that the output correctly has 2
}

// Scala Code: Calling Driver to instantiate Simple, SimpleTester, and execute the test
// Don't worry about understanding this code, it is very complicated Scala
val works = Driver(() => new Simple) {
  c => new SimpleTester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 7.3 ms
Total FIRRTL Compile Time: 6.3 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1503431103220
test cmd6WrapperHelperSimple Success: 3 tests passed in 5 cycles taking 0.001590 seconds
[info] [0.001] RAN 0 CYCLES PASSED
SUCCESS!!


defined class SimpleTester
works: Boolean = true

## Understanding Chisel vs Scala

Earlier, we compiled our Chisel into Verilog. It is important to understand what this process is, so that you as a designer can more clearly differentiate between what is hardware (Chisel) and what is scripting/parameterizations (Scala).

Elaboration is the process of constructing hardware when running a Scala/Chisel program. This hardware could be represented as a schematic, or a Verilog file with neither parameters nor generate statements. In essence, elaboration removes all scripting/parameterizing around a circuit, leaving only the bare description of hardware.

Let's look at some examples which clarify ***scripting*** values from ***hardware*** values.

In this first example, we have two Modules `ScalaPlus` and `VerilogPlus`. In both examples, we call the `+` operator. However, the first `+` is resolved during elaboration (`1 + 2 == 3`), while the second instantiates a Verilog `+` operator and explicitly trims the carry-out. *Note that the Verilog now contains many temporary nodes of the form* `_T_<#>`*.*

In [34]:
// Chisel Code: Declare a new module definition
class ScalaPlus extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  io.out := (1 + 2).U  // `+` resolved during elaboration
}
println(getVerilog(new ScalaPlus))

class VerilogPlus extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  io.out := 1.U + 2.U  // `+` remains in Verilog
}
println(getVerilog(new VerilogPlus))

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 4.9 ms
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif

module cmd33WrapperHelperScalaPlus(
  input        clock,
  input        reset,
  output [1:0] io_out
);
  assign io_out = 2'h3;
endmodule

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 4.6 ms
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif

module cmd33WrapperHelperVerilogPlus(
  input        clock,
  input        reset,
  output [1:0] io_out
);
  wire [2:0] _T_5;
  wire [1:0] _T_6;
  assign io_out = _T_6;
  assign _T_5 = 2'h1 + 2'h2;
  assign _T_6 = _T_5

defined class ScalaPlus
defined class VerilogPlus

***What is going on here???*** When we elaborate a Chisel design, we are executing the Scala code that implements Chisel. It is designed in such a way that when it executes, it builds up an un-parameterized hardware circuit.

In `ScalaPlus`:
- <span style="color:red; font-family:monospace">(1 + 2)</span> is an evaluated during elaboration, resulting in `3`
- `(1 + 2)`<span style="color:red; font-family:monospace">.U</span> converts `3` into a 2-bit hardware constant, represented by Verilog's `2'h3`
- `out `<span style="color:red; font-family:monospace"> := </span>` (1 + 2).U` combinationally connects `3.U` to `out`, resulting in the Verilog `assign out = 2'h3;`.

In `ChiselPlus`:
- `1`<span style="color:red; font-family:monospace">.U</span> creates a 1-bit hardware constant, represented by Verilog's `1'h1`
- `2`<span style="color:red; font-family:monospace">.U</span> creates a 2-bit hardware constant, represented by Verilog's `2'h2`
- `1.U ` <span style="color:red; font-family:monospace">+</span> ` 2.U` creates a non-expanding hardware adder, represented by Verilog's `+` operator with logic to omit the carryout
- `out `<span style="color:red; font-family:monospace"> := </span> `1.U + 2.U` creates a combinational connection between the result of `+` with `out`, resulting in the Verilog `assign out = ...` statement



## Combinational Logic
### Data Types

Chisel adds a set of hardware types to the Scala type system. Chisel types eventually map to wires and logic. Example declarations of Chisel literals and variables in various types are given below. `B` stands for boolean, which is functionally the same as a one-bit `UInt`. `U` stands for `UInt` or unsigned integer. `S` stands for `SInt` or signed integer. Note that the width of an `SInt` includes the sign bit, so a 4-bit `SInt` ranges from -8 to 7. Notice how all Chisel variables are Scala `val`s. Never use a Scala `var` for a hardware construct, since the construct may never change once defined; only its value may change.
```scala
// literals or constants:
val foo = 4.U(8.W)
val bar = -4.S(11.W)
val qux = true.B
    
// variables or wires:
val foo2 = UInt(8.W)
val bar2 = SInt(11.W)
val qux2 = Bool()
```

### Chisel Operators

An overview of Chisel operators is given below. For a more complete list, consult the [Chisel3 cheatsheet](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf), specifically the third column on the first page.

```scala
// Colon equals assigns the value of the expression on the right-hand side to the variable on the left-hand side
val foo = UInt(16.W)        // define a wire called foo of type unsigned integer with width 16 bits
foo := io.a ^ io.b          // foo is assigned a XOR b (bitwise)
io.z := foo                 // assigns foo to the output z
    
// Comparisons return a Chisel Bool type, NOT a Scala Boolean type!!
val bar = io.a === io.b     // bar is 1 if a equals b, otherwise 0
val qux = io.a =/= io.b     // qux is 1 if a does not equal b, otherwise 0
val baz = io.a  >  io.b     // baz is 1 if a is greater than b, otherwise 0 (see also <, >=, and <=)
 
// Bit-level and ternary operators 
val corge = io.a(10, 4)     // extracts bits 4, 5, 6, 7, 8, and 9 from the 16-bit input a (0 indexed)
val waldo = io.a(15)        // extracts bit 15 from the 16-bit input a (the MSB)
val plugh = Cat(io.a, io.b) // concatenates the two 16-bit inputs a and b into plugh, a 32-bit UInt
val xyzzy = Mux(io.a < io.b, io.a, io.b) // assigns to xyzzy input a if a is less than b, otherwise input b
  
// Math operators
val quux = io.a + io.b      // assigns to quux the sum of a and b; the width of quux is inferred
val fred = io.a - io.b      // assigns to fred the difference between a and b; the width of fred is inferred
val thud = io.a * io.b      // assigns to thud the product of a and b
```

Now that you have a cursory overview of operators, let's practice using them and data types with the following exercise. Write code that passes the test. First scroll down and view the tester, then modify the OperatorsExercise module in the indicated locations. Remember our goal of producing an LFSR. The input is a 4-bit word that we'll later store in a shift register. The result is the output of the LFSR (which will be fed back into the input of the shift register). Your task is to make the logic for an LFSR of the polynomial x<sup>4</sup>+x<sup>3</sup>+1. Note that the 1 is implicit, so bit 0 of the input is really x<sup>1</sup>.

Graphically, the circuit looks like:

![LFSR-4 combinational](images/lfsr4combinational.svg)

In [ ]:
// Create the module
class OperatorsExercise extends Module {
  val io = IO(new Bundle {
    val input = Input(UInt(4.W))
    val result = Output(Bool())
  })

  // YOUR CODE GOES HERE
  // Remember: input and result are inside io, so access them with io.input and io.result
  // Also, there's no need to typecast between 1-bit UInts and Bools
}

// Create the test
class OperatorsExerciseTester(c: OperatorsExercise) extends PeekPokeTester(c) {
  for (i <- 0 until math.pow(2,4).toInt) {
    poke(c.io.input, i)
    expect(c.io.result, (i >> 2 & 1) ^ (i >> 3 & 1))
  }
}

// Run the test!
val works = Driver(() => new OperatorsExercise) {
  c => new OperatorsExerciseTester(c)
}
println(works)

## Sequential Logic

_In the previous section, we built the combinational part of the LFSR - that is, given the simultaneous output of all the shift register bits, generate the bit to feed back in. In this section, we will build the sequential part, the shift registers themselves._

The basic stateful element in Chisel is the register, or `Reg`. It keeps its current value through a clock cycle, and can optionally be updated with a new value that becomes visible at the next clock edge.

_Chisel connects an implicit clock line to all state elements, and an implicit reset line to state elements with an _ initial _ value. There are constructs for overriding this, for example in multi-clock designs, but we won't go into those here._

A register can be constructed given a data type. For instance, a 2-bit unsigned integer register without initialization:
```scala
val myReg = Reg(UInt(2.W))
```

or with initialization (to 1 decimal). Both versions below are valid and do the same thing:
```scala
val myReg = RegInit(UInt(2.W), 1.U)
val myReg = RegInit(1.U(2.W))
```

Its value can be updated with the `:=` operator. For example, this increments the register's value:
```scala
myReg := myReg + 1.U
```

Note that when read, it gives its current (stored) value, but the assignment doesn't take effect until the next clock edge.

Below is an example which uses a register to implement a 2-bit wide counter, initializing at 1 and overflowing from 3 to zero. Run the block and take a look at the output trace.

![LFSR-4](images/counter2.svg)

In [ ]:
// Module containing the register
class MyCounter extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)
  myReg := myReg + 1.U
  io.out := myReg

  
  printf("out=%d\n", io.out)
  
  printf(p"${io}\n")
}

// Testvector
class MyCounterTester(c: MyCounter) extends PeekPokeTester(c) {
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 3)
  step(1)
  expect(c.io.out, 0)
  step(1)
  expect(c.io.out, 1)
  step(1)
}

// Driver run invocation
Driver(() => new MyCounter) {
  c => new MyCounterTester(c)
}

One important note is that Chisel distinguishes between types (like `UInt`) and hardware nodes (like the literal `2.U`, or the output of `myReg`). While
```scala
val myReg = Reg(UInt(2.W))
```
is legal because a Reg needs a data type as a model,
```scala
val myReg = Reg(2.U)
```
is an error because `2.U` is already a hardware node and can't be used as a model.

Similarly, the `RegInit` examples have a type as their first argument and hardware node as the second argument. The hardware node is required to give a concrete initialization value.

Given that, now build a module that implements a shift register for your LFSR. Specifically:
- Each element is a single bit wide.
- Has 4 stages.
- Takes a single input bit, which is the next value into the shift register.
- Outputs the parallel output of the shift register, with the most significant bit being the last element of the shift register and the least significant bit being the first element of the shift register. `Cat` may come in handy.
- The output initializes at `b0001`.
- Shifts each clock cycle (no enable signal).

![4-stage shifter](images/shifter4.svg)

A basic Module skeleton, testvector, and Driver invocation is provided below. The first register has been provided for you.

In [ ]:
class MyShiftRegister extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(UInt(4.W))
  })
  val s1 = RegInit(Bool(), true.B)
    
  // YOUR CODE HERE
  // don't forget to update the first register!
  
  printf("%x\n", io.out)
}

In [ ]:
class MyShiftRegisterTester(c: MyShiftRegister) extends PeekPokeTester(c) {
  expect(c.io.out, 1)  // b0001
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 2)  // b0010
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 5)  // b0101
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 11)  // b1011
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 6)  // b0110
}

Driver(() => new MyShiftRegister) {
  c => new MyShiftRegisterTester(c)
}

Chisel also offers several more variants of the register constructor, for example with a `next` field or with an `enable` signal, but these are outside the scope of this introductory tutorial.

## Conditional Logic and Wires
_We've now seen the basics of building combinational and sequential logic circuits, but those are very low-level primitives. Chisel provides a higher level of abstraction for some operations._

### Conditional Register Updates
Chisel operators can be gated by putting them inside a `when` conditional block. In our counter example above, if we also added a `count` input signal, we could gate the incrementing of the counter as follows:

```scala
when (io.count) {
  myReg := myReg + 1.U
}
```

In the case the `when` condition isn't met, the code inside simply isn't "run" on that cycle. Registers hold their state by default, so `myReg` will "pause" when the `when` condition isn't met, unless it is assigned later on to a different value.

When also has an `.elsewhen` and `.otherwise` construct, similar to `else if` and `else`. For example, if we wanted to count the number of cycles `count` is consecutively high, we could have it reset in the `.otherwise` condition:

```scala
when (io.count) {
  myReg := myReg + 1.U
} .otherwise  {
  myReg := 0.U
}
```

_Note that there is a `.` before the `otherwise`, this due to limitations of Scala syntax._

### Wires
While we've seen the update operator `:=` used combinationally on Module boundary `IO`s, it can be also applied to Module-internal `Wire`s. Here's an example of the counter above, styled slightly differently to make the combinational logic explicit and using `Wire`s.
```scala
val myReg = RegInit(UInt(2.W), 1.U)
val nextReg = Wire(UInt(2.W))  // constructed similarly to Regs
nextReg := myReg + 1.U
myReg := nextReg
```

### Conditional Combinational Updates
Conditional blocks can also be used to generate combinational logic. Let's say now that we want a counter that counts 0, 1, 2, then resets. In this case, since we need to reset on 2, we can no longer depend on the implicit integer overflow and need to make the logic explicit:
```scala
val nextReg = Wire(UInt(2.W))
when (myReg === 2.U) {
  nextReg := 0.U
} .otherwise {
  nextReg := myReg + 1.U
}
```

When there are multiple update operations that can apply at the same time, the last one takes priority. So the above example can be equivalently rewritten by unconditionally updating `nextReg` with the counter increment, then overriding it in the overflow case:

```scala
val nextReg = Wire(UInt(2.W))
nextReg := myReg + 1.U
when (myReg === 2.U) {
  nextReg := 0.U
}
```

There is also `WireInit`, a shorthand which declares and initializes a `Wire`:

```scala
val nextReg = WireInit(UInt(2.W), myReg + 1.U)
when (myReg === 2.U) {
  nextReg := 0.U
}
```

Whether to use the first, second, or third forms is mainly a matter of style and readability. They are all functionally equivalent.

### A Gated, 3-tick Counter
Putting it all together, here's the full example:

In [ ]:
// Module containing the register
class MyNewCounter extends Module {
  val io = IO(new Bundle {
    val count = Input(Bool())
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)

  val nextReg = Wire(UInt(2.W))
  when (myReg === 2.U) {
    nextReg := 0.U
  } .otherwise {
    nextReg := myReg + 1.U
  }
  
  when (io.count) {
    myReg := nextReg
  }

  io.out := myReg
    
  printf("out=%d\n", io.out)
}

// Testvector
class MyNewCounterTester(c: MyNewCounter) extends PeekPokeTester(c) {
  poke(c.io.count, 1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 0)
  step(1)
  expect(c.io.out, 1)
  poke(c.io.count, 0)  // test pause
  step(1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 1)
  poke(c.io.count, 1)  // unpause
  step(1)
  expect(c.io.out, 2)
  step(1)
}

// Driver run invocation
Driver(() => new MyNewCounter) {
  c => new MyNewCounterTester(c)
}

The `when` blocks are high-level constructs that ease the generation of muxes. The actual circuit would look something like:

![Counter 3](images/counter3.svg)

Take your shift register design from above, but modify it so that it only shifts when the update signal is asserted.

As with before, a basic testbench and module skeleton are provided for you.

In [ ]:
class MyGatedShiftRegister extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val update = Input(Bool())
    val out = Output(UInt(4.W))
  })
  val s1 = RegInit(Bool(), true.B)
    
  // YOUR CODE HERE
  // don't forget to update the first register!
  
  printf("%x\n", io.out)  
}

In [ ]:
class MyGatedShiftRegisterTester(c: MyGatedShiftRegister) extends PeekPokeTester(c) {
  poke(c.io.update, 1)
  expect(c.io.out, 1)  // b0001
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 2)  // b0010
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 5)  // b0101
  poke(c.io.update, 0)  // pause
  step(1)
  expect(c.io.out, 5)
  step(1)
  expect(c.io.out, 5)
  step(1)
  poke(c.io.update, 1)  // unpause
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 11)  // b1011
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 6)  // b0110
}

Driver(() => new MyGatedShiftRegister) {
  c => new MyGatedShiftRegisterTester(c)
}

## Modules
_Without re-use, any non-trivial RTL design wouldn't get very far. This section goes into the basics of re-use with Chisel._

In all the example code, you've probably noticed that all your classes extend the `Module` class. Chisel `Module`s are very much like Verilog `module`s, they are instantiable blocks of logic that contain well-defined IO ports.

As all `Module`s are classes, they can be instantiated using the `new` object syntax. However, they also must be wrapped in a `Module(...)` call. For example, to instantiate the 4-state counter example way above:
```scala
val myModule = Module(new MyCounter())
```

You can then reference its IO objects in its parent class. For example, if you wanted to know when `myModule`'s count was 3, you could:
```scala
val isCount3 = myModule.io.out === 3.U
```

### A 4-cycle tick
This example below uses the `MyCounter` class from above to build a block that generates a one high cycle for every 4 clock cycles by comparing its output.

![4-ticker](images/myTicker.svg)

In [ ]:
// Module containing the register
class MyTicker extends Module {
  val io = IO(new Bundle {
    val out = Output(Bool())
  })
  
  val myModule = Module(new MyCounter())
  io.out := myModule.io.out === 3.U
}

// Testvector
class MyTickerTester(c: MyTicker) extends PeekPokeTester(c) {
  expect(c.io.out, 0)  // counter at 1
  step(1)
  expect(c.io.out, 0)  // counter at 2
  step(1)
  expect(c.io.out, 1)  // counter at 3
  step(1)
  expect(c.io.out, 0)  // counter at 0
  step(1)
  expect(c.io.out, 0)  // counter at 1
  step(1)
  expect(c.io.out, 0)  // counter at 2
  step(1)
  expect(c.io.out, 1)  // counter at 3
  step(1)
}

// Driver run invocation
Driver(() => new MyTicker) {
  c => new MyTickerTester(c)
}

Modules are just one form of composition and re-use in Chisel, you'll learn more soon.

## Now You Try: The Full LFSR

Now, build a full LFSR by taking your `MyGatedShiftRegister` and `OperatorsExercise` modules and instantiating them in a parent module. As usual, the module skeleton and a testbench is provided for you.

![LFSR-4](images/lfsr4blocks.svg)

In [ ]:
class MyLfsr extends Module {
  val io = IO(new Bundle {
    val update = Input(Bool())
    val out = Output(UInt(4.W))
  })
  
  val myShifter = Module(new MyGatedShiftRegister())
  myShifter.io.update := io.update
  
  // YOUR CODE HERE
  // Don't forget to make the other needed connections to myShifter!
  
  printf("%x\n", io.out)
}


In [ ]:
// Testvector
class MyLfsrTester(c: MyLfsr) extends PeekPokeTester(c) {
  poke(c.io.update, 1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 4)
  step(1)
  expect(c.io.out, 9)
  step(1)
  expect(c.io.out, 3)
  step(1)
  expect(c.io.out, 6)
  step(1)
  expect(c.io.out, 13)
  step(1)
  expect(c.io.out, 10)
  step(1)
  expect(c.io.out, 5)
  step(1)
  expect(c.io.out, 11)
  poke(c.io.update, 0)  // pause
  step(1)
  expect(c.io.out, 11)
  step(1)
  expect(c.io.out, 11)
  poke(c.io.update, 1)  // unpause
  step(1)
  expect(c.io.out, 7)
  step(1)
  expect(c.io.out, 15)
  step(1)
  expect(c.io.out, 14)
  step(1)
  expect(c.io.out, 12)
  step(1)
  expect(c.io.out, 8)
  step(1)
  expect(c.io.out, 1)  // LFSR restarts
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 4)
  step(1)
  expect(c.io.out, 9)
  step(1)
}

// Driver run invocation
Driver(() => new MyLfsr) {
  c => new MyLfsrTester(c)
}

## Scala vs. Chisel

## Abstraction and Hierarchy in Hardware

TODO: Include `Mem` and `BlackBox` here (`Mem` = shift register `Mem`, BlackBox = maybe a negative-edge FF, or reimplementation of OperatorsExercise in Verilog??)

TODO: Also include `Bundle` (as both IO inside Module and as separate class) and `Vec`

## Debugging Tips
TODO: add more, explain what's going on below (string interpreter)
TODO: maybe give quick overview of FIRRTL and using it as a debugging technique

In [ ]:
// Module containing the register
class MyCounter extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)
  myReg := myReg + 1.U
  io.out := myReg

  // printing results
  printf("out=%d\n", io.out)
  printf(p"${io}\n")
}

In [ ]:
// viewing the firrtl for debugging:
println(chisel3.Driver.emit(() => new MyTicker))

In [31]:
// Chisel Code: Declare a new module definition
class Simple extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

// Scala Code: Elaborate our Chisel design into FIRRTL by calling Driver.emit
val firrtlString = chisel3.Driver.emit(() => new Simple)
println(firrtlString)

[info] [0.000] Elaborating design...
[info] [0.001] Done elaborating.
;buildInfoPackage: chisel3, version: 3.0-SNAPSHOT_2017-07-19, scalaVersion: 2.11.11, sbtVersion: 0.13.15, builtAtString: 2017-07-19 18:56:34.453, builtAtMillis: 1500490594453
circuit cmd30WrapperHelperSimple : 
  module cmd30WrapperHelperSimple : 
    input clock : Clock
    input reset : UInt<1>
    output io : {flip in : UInt<4>, out : UInt<4>}
    
    clock is invalid
    reset is invalid
    io is invalid
    io.out <= io.in @[cmd30.sc 6:10]
    



defined class Simple
firrtlString: String = """
;buildInfoPackage: chisel3, version: 3.0-SNAPSHOT_2017-07-19, scalaVersion: 2.11.11, sbtVersion: 0.13.15, builtAtString: 2017-07-19 18:56:34.453, builtAtMillis: 1500490594453
circuit cmd30WrapperHelperSimple : 
  module cmd30WrapperHelperSimple : 
    input clock : Clock
    input reset : UInt<1>
    output io : {flip in : UInt<4>, out : UInt<4>}
    
    clock is invalid
    reset is invalid
...